In [2]:
import pandas as pd
import numpy as np
!pip install seaborn
import seaborn as sns


  Using cached seaborn-0.10.1-py3-none-any.whl (215 kB)
  Using cached matplotlib-3.3.1-cp38-cp38-manylinux1_x86_64.whl (11.6 MB)
  Using cached scipy-1.5.2-cp38-cp38-manylinux1_x86_64.whl (25.7 MB)
  Using cached kiwisolver-1.2.0-cp38-cp38-manylinux1_x86_64.whl (92 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached Pillow-7.2.0-cp38-cp38-manylinux1_x86_64.whl (2.2 MB)


In [4]:
!pip install tweepy
import tweepy                   # Python wrapper around Twitter API
from datetime import date
from datetime import datetime
import time
import keys

In [5]:
api_key = keys.api_key
api_secret_key = keys.api_secret_key
access_token = keys.access_token
access_token_secret = keys.access_token_secret
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


In [6]:
def limit_handled(cursor, list_name):
    """Function to catch the tweepy api rate limit"""
    while True:
        try:
            yield cursor.next()    # Catch Twitter API rate limit exception and wait for 15 minutes
        except tweepy.RateLimitError:
            print("\nData points in list = {}".format(len(list_name)))
            print('Hit Twitter API rate limit.')
            for i in range(3, 0, -1):
                print("Wait for {} mins.".format(i * 5))
                time.sleep(5 * 60)    # Catch any other Twitter API exceptions
        except tweepy.error.TweepError:
            print('\nCaught TweepError exception' )
        except StopIteration:
            break
            
#function to get tweets
def get_all_tweets_info(handle):
    """Gets the follwing information from a twitter account number of likes,number of retweets,number of tweets"""
    alltweets = []  
    new_tweets = api.user_timeline(screen_name = handle,count=200)
    alltweets.extend(new_tweets)
    oldest = alltweets[-1].id - 1 
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))    # all subsequent requests use the max_id param to prevent
        # duplicates
        new_tweets = api.user_timeline(screen_name = handle,count=200,max_id=oldest)    # save most recent tweets
        alltweets.extend(new_tweets)    # update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        print("...%s tweets downloaded so far" % (len(alltweets)))
    tweets_list = []
    likes = []
    retweet_count = []
    
    for tweet in alltweets:
        tweets_text = tweet.text
        tweets_likes = tweet.favorite_count
        tweets_retweeted = tweet.retweeted

        
        tweets_list.append(tweets_text)
        likes.append(tweets_likes)
        retweet_count.append(tweets_retweeted)
        
    return len(tweets_list),sum(likes),sum(retweet_count)
    
def followers_count(handle):
    """ Determines the number of followers for a given twitter account"""
  
    followers_list = []
    cursor = tweepy.Cursor(api.followers,screen_name=handle,count=200).pages()
    print("loading followers number")
    for i, page in enumerate(limit_handled(cursor, followers_list)):  

        # Add latest batch of follower data to the list
        followers_list += page
    follower_count = len(followers_list)
    return follower_count

def following_count(handle):
    """Determines the number of people following a given twitter account"""
    following_list = []
    cursor = tweepy.Cursor(api.friends,screen_name=handle,count=200).pages()
    print("Loading following numbers")
    for i,page in enumerate(limit_handled(cursor, following_list)):
        following_list += page
    friends_count = len(following_list)
    return friends_count 

In [7]:
tweets_count,likes_count,retweet_count = get_all_tweets_info('okiomagerald')
fllwing_count = followers_count('okiomagerald')
follow_c = following_count('okiomagerald')

getting tweets before 1126970263160139775
...395 tweets downloaded so far
getting tweets before 787330835582160895
...415 tweets downloaded so far
getting tweets before 579974579231260671
...415 tweets downloaded so far
loading followers number
Loading following numbers


In [8]:
print(tweets_count)
print(likes_count)
print(retweet_count)

415
465
166


In [9]:
print(fllwing_count)
print(follow_c)

214
296


In [16]:
def popularity_score(retweet_count,likes_count):
    pop_score = retweet_count + likes_count
    print(pop_score)
    return pop_score

def reach_score(followers_count,following_count):
    score = followers_count - following_count
    print(score)
    return score

def relevance_score(comments_count,mentions_count):
    rel_score = comments_count + mentions_count
    print(rel_score)
    return rel_score

In [17]:
popularity_score(tweets_count,likes_count)

880


880

In [ ]:
#dataframe for our data
data_tweets = pd.DataFrame(columns=['Handle','N_tweets','N_likes','N_retweeted','N_followed','N_following'])